In [1]:
import os
os.system('color')
import random
from termcolor import colored
from typing import List, Optional
from copy import deepcopy
import csv
from enum import Enum

import tensorflow as tf
from keras import layers, Model
import numpy as np
from sklearn.preprocessing import OneHotEncoder
# import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator

import utility as ut

os.system('color')
random.seed(0)
np.random.seed(0)

DATAPATH = os.path.join(os.getcwd(), "newdatasets")
RESULTS_PATH = os.path.join(os.getcwd(), "autoencoder_results")

sh: 1: color: not found
2023-08-17 08:45:58.808426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 08:45:58.939577: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-17 08:45:58.965060: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-17 08:45:59.457967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic lib

In [7]:
'''
Trial summary
Hyperparameters:
conv1_filter_size: 10
conv2_filter_size: 9
conv1_filters: 50
conv2_filters: 66
bridge_filters: 120
max_pool: 2
dropout_: : 0.35
Score: 0.9727120995521545


Hyperparameters:
conv1_filter_size: 5
conv2_filter_size: 9
conv1_filters: 46
conv2_filters: 90
bridge_filters: 160
bridge_filter_size: 5
max_pool: 2
dropout_: : 0.5
Score: 0.9742878675460815
'''

# =============================================================================
class Encoder(layers.Layer):
    """Encoder part of autoencoder"""

    # -------------------------------------------------------------------------
    def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=10, conv2_filter_size=9, maxpool=2, dropout=0.25, name="encoder", **kwargs):
        super().__init__(name=name, **kwargs)

        self.conv1 = layers.Conv1D(conv1_filters, conv1_filter_size, padding="same", activation="relu")
        self.conv2 = layers.Conv1D(conv2_filters, conv2_filter_size, padding="same", activation="relu")
        self.maxpool = layers.MaxPooling1D(maxpool, padding="same")
        self.dropout = layers.Dropout(dropout)
        

    # -------------------------------------------------------------------------
    def call(self, inputs):
        return self.dropout(self.maxpool(self.conv2(self.dropout(self.maxpool(self.conv1(inputs))))))

# =============================================================================
class Decoder(layers.Layer):
    """Decoder part of autoencoder"""

    # -------------------------------------------------------------------------
    def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=5, conv2_filter_size=9, maxpool=2, dropout=0.25, name="decoder", **kwargs):
        super().__init__(name=name, **kwargs)

        self.conv1 = layers.Conv1DTranspose(conv1_filters, conv1_filter_size, padding="same", activation="relu")
        self.conv2 = layers.Conv1DTranspose(conv2_filters, conv2_filter_size, padding="same", activation="relu")
        self.upsample = layers.UpSampling1D(maxpool)
        self.dropout = layers.Dropout(dropout)

    # -------------------------------------------------------------------------
    def call(self, inputs):
        return self.dropout(self.upsample(self.conv2(self.dropout(self.upsample(self.conv1(inputs))))))

# =============================================================================
class Autoencoder(Model):
    """Autoencoder"""
    # -------------------------------------------------------------------------
    def __init__(self, num_classes, name="autoencoder", **kwargs):
        super().__init__(name=name, **kwargs)

        self.num_classes = num_classes
        self.hyperparameters = {
            "conv1_filters": 46,
            "conv2_filters": 90,
            "conv1_filter_size": 5,
            "conv2_filter_size": 9,
            "bridge_filters": 160,
            "bridge_filter_size": 5,
            "dropout": 0.50,
            "maxpool": 5,
        }

        hp = self.hyperparameters

        self.encoder = Encoder(hp["conv1_filters"], hp["conv2_filters"])
        self.bridge = layers.Conv1D(hp["bridge_filters"], hp["bridge_filter_size"], padding="same", activation="relu")
        self.decoder = Decoder(hp["conv2_filters"], hp["conv1_filters"])
        self.finallayer = layers.Conv1D(self.num_classes, hp["conv1_filter_size"], padding="same", activation="softmax")
    # -------------------------------------------------------------------------
    def call(self, inputs):
        return self.finallayer(self.decoder(self.bridge(self.encoder(inputs))))
# =============================================================================
class PaddingType(Enum):
    ZERO=1
    TRUNCATE=2
    EMPTY=3

# =============================================================================
class NoisificationMethod(Enum):
    RANDOMSCATTER=1
    RANDOMCONTIG=2

weights = None
# =============================================================================
class ProteinScaffoldFixer():
    """Class to correct errors in a protein scaffold and fill gaps"""

    # -------------------------------------------------------------------------
    def __init__(self, 
                output_seqs, 
                paddingtype=PaddingType.EMPTY, 
                noise_percent=0.2, 
                noisemethod=NoisificationMethod.RANDOMSCATTER, 
                numgaps=5, 
                mingapsize=3, 
                mincontigsize=1, 
                epochs=300,
                optimizer="adam",
                early_stopping=False):

        self.epochs = epochs
        self.optimizer = optimizer
        self.paddingtype = paddingtype
        self.noise_percent = noise_percent
        self.early_stopping = early_stopping

        self.classes = np.array(["-", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z'])
        self.ohe = OneHotEncoder(sparse_output=False, categories=[self.classes])

        self.max_seq_length = max((len(seq) for seq in output_seqs))
        self.maxpool = 2
        while self.max_seq_length % (self.maxpool * self.maxpool) != 0:
            self.max_seq_length += 1

        self.output_seqs = output_seqs
        self.input_seqs = self.noisify_sequences(output_seqs, noise_percent, noisemethod, numgaps, mingapsize, mincontigsize)

        self.train_x = self.preprocess_sequences(self.input_seqs)
        self.train_y = self.preprocess_sequences(self.output_seqs)

        #self.history = None

        self.autoencoder = Autoencoder(len(self.classes))

    # =============================================================================
    def noisify_sequences(self, seqs, noise_percent, noisemethod, numgaps=5, mingapsize=3, mincontigsize=3):
        if noisemethod == NoisificationMethod.RANDOMSCATTER:
            return self.noisify_by_random_scatter(seqs, noise_percent)
        elif noisemethod == NoisificationMethod.RANDOMCONTIG:
            return self.noisify_by_random_contigs(seqs, noise_percent, numgaps, mingapsize, mincontigsize)

    # =============================================================================
    def noisify_by_random_scatter(self, seqs, noise_percent):
        sequences = deepcopy(seqs)
        # To noisify our input data, we will replace random amino acids with something else
        for seq in sequences:

            # We randomly sample from all the possible indices of seq
            indices_to_replace = random.sample(range(len(seq)), int(noise_percent * len(seq)))
            seq[indices_to_replace] = "-"

        return sequences

    # =============================================================================
    def noisify_by_random_contigs(self, seqs, noise_percent, numgaps, mingapsize, mincontigsize):

        sequences = deepcopy(seqs)
        for seqind, seq in enumerate(sequences):

            amino_acid_length = len(seq)
            amino_acids_to_replace = int(amino_acid_length * noise_percent)

            # The idea is to build a gap queue, randomly putting each amino acid to replace into each gap
            # Then, we build a contig queue, randomly putting each amino acid into each contig
            gap_queue = np.zeros(numgaps).astype(int)
            for _ in range(amino_acids_to_replace):

                # Any gap can be considered
                valid_gaps = np.arange(numgaps)

                # ... so long as we have no underfilled gaps
                underfilled = gap_queue < mingapsize
                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_gaps = valid_gaps[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                gap_queue[np.random.choice(valid_gaps)] += 1

            # There can always be one more contig than gaps (if there's a contig at beginnning and end of sequence)
            contig_queue = np.zeros(numgaps+1).astype(int)

            # We have to allocate all the amino acids NOT in gaps into contigs BETWEEN the gaps
            for _ in range(amino_acid_length - amino_acids_to_replace):

                # Any contig can be considered
                valid_contigs = np.arange(len(contig_queue))

                # ... so long as we have no underfilled contigs
                underfilled = contig_queue < mincontigsize

                # The exceptions are the first and last contigs. They are never considered underfilled
                underfilled[0] = False
                underfilled[-1] = False

                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_contigs = valid_contigs[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                contig_queue[np.random.choice(valid_contigs)] += 1
            
            # Once we have determined gap_queue and contig_queue, we iterate over them to set the gaps equal
            # to our blank amino acid character

            sequence_pointer = 0
            iscontig=True
            while sequence_pointer < len(seq):
                if iscontig:
                    # Don't do anything for the contig except increment the pointer and pop off the contig queue
                    sequence_pointer += contig_queue[0]
                    contig_queue = np.delete(contig_queue, 0)
                else:
                    seq[sequence_pointer:sequence_pointer+gap_queue[0]] = '-'
                    sequence_pointer += gap_queue[0]
                    gap_queue = np.delete(gap_queue, 0)

                # We alternate between contigs and gaps
                iscontig = not iscontig

        return sequences

    # -------------------------------------------------------------------------
    def predict_sequence(self, seq, predict_only_gaps):

        scaffold = self.preprocess_sequences([seq])
        pred = self.autoencoder.predict(scaffold).reshape(self.max_seq_length, len(self.classes))

        # Set the probability of empty "-" to zero, since we always want to predict something
        emptyclass = np.where(self.ohe.transform(np.array("-").reshape(-1, 1))[0])[0][0]
        pred[:, emptyclass] = 0.0

        # Convert the probability distribution to a one-hot encoded vector
        mask = pred == np.amax(pred, axis=1).reshape(pred.shape[0], 1)
        indices = list((i, np.where(mask[i])[0][0]) for i in range(mask.shape[0]))

        pred = np.zeros(pred.shape)
        for i in indices:
            pred[i] = 1

        # Then we can use our one hot encoder to convert back to the original sequence of classes
        pred = self.ohe.inverse_transform(pred[:len(seq), :len(self.classes)]).reshape(len(seq))

        if predict_only_gaps:
            # We only care about predicting the gaps in seq, so replace amino acids in prediction with original nongaps
            nongaps = np.where(seq != '-')[0]
            pred[nongaps] = seq[nongaps]

        return pred

    # =============================================================================
    def preprocess_sequences(self, seqs: List[np.array]) -> np.array:

        # the value -1 lets numpy know to infer the shape. So it's just a column vector of length num_samples
        seqs = [np.array(seq).reshape(-1, 1) for seq in seqs]

        # One-hot encode each sequence
        seqs = [self.ohe.fit_transform(seq) for seq in seqs]

        if self.paddingtype == PaddingType.ZERO:
            # The sequences may have different lengths, so we will pad them with zeros
            # We are padding to fill up to max length, then we can turn it into a single numpy tensor
            return np.array([np.pad(seq, ((0, self.max_seq_length-len(seq)), (0, 0))) for seq in seqs])

        elif self.paddingtype == PaddingType.EMPTY:

            # We pad with the "empty" class
            emptyclass = self.ohe.fit_transform(np.array("-").reshape(-1, 1))[0]

            # dynamically extend each seq by enough emptyvals to make a single sequence length
            return np.array([np.vstack((seq, *(emptyclass for _ in range(self.max_seq_length-len(seq))))) for seq in seqs])

        elif self.paddingtype == PaddingType.TRUNCATE:
            # FIXME: Implement me
            raise Exception("Have not implemented Padding Type TRUNCATE!")

    # =============================================================================
    def train(self, verbose="auto", optimizer="adam", epochs=None):
        self.optimizer = optimizer
        if epochs:
            self.epochs = epochs

        self.autoencoder.compile(optimizer=self.optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        callbacks = []
        if self.early_stopping:
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10))
        self.history = self.autoencoder.fit(self.train_x, self.train_y, epochs=self.epochs, validation_split=0.15, verbose=verbose, callbacks=callbacks)
        print(self.autoencoder.summary())
        
# =============================================================================
def highlight_indices(seq: np.array, indices: np.array, color: str):
    # We use deepcopy to prevent mutation and we cast to object so that we can treat contents as python strings
    # otherwise, it gets messed up as it treats each element as a single character
    newseq = deepcopy(seq).astype('object')
    newseq[indices] = np.vectorize(lambda x: colored(x, color, attrs=["bold"]))(seq[indices])
    return newseq

# =============================================================================
def print_sequence(seq, 
                   header: str=None, 
                   incorrect_indices: Optional[np.array]=None, 
                   correct_indices: Optional[np.array]=None):

    newseq = deepcopy(seq)
    if correct_indices is not None and correct_indices.size != 0:
        newseq = highlight_indices(newseq, correct_indices, "green")
    if incorrect_indices is not None and incorrect_indices.size != 0:
        newseq = highlight_indices(newseq, incorrect_indices, "red")

    line_length = 40
    if header:
        print(header)
    print("=" * line_length)

    i = 0
    while i < len(newseq):
        print(" ".join(newseq[i: i+line_length]))
        i += line_length

# =============================================================================
def get_sequences(fasta_file: str) -> List[np.array]:
    sequences = []
    lines = []
    with open(fasta_file, "r") as input_file:
        lines = list(filter(None, input_file.read().split("\n")))

    parts = []
    for line in lines:
        if line.startswith(">"):
            if parts:
                sequences.append(np.array([c for c in "".join(parts)]))
            parts = []
        else:
            parts.append(line)
    if parts:
        sequences.append(np.array([c for c in "".join(parts)]))
    return sequences

# =============================================================================
def snake_case_prettify(s):
    return " ".join(w.capitalize() for w in s.split("_"))

# =============================================================================
def save_models(models):
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if not os.path.exists(modeldir):
        os.makedirs(modeldir)

    for name, model in models.items():
        model.save(os.path.join(modeldir, name))

# =============================================================================
def load_models():
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if os.path.exists(modeldir):
        return {f: tf.keras.models.load_model(os.path.join(modeldir, f)) for f in os.listdir(modeldir)}
    return {}

# =============================================================================
def print_diff_between_target_and_de_novo():

    dataset_dir = os.path.join(os.getcwd(), "newdatasets")
    
    de_novo_sequence = get_sequences(os.path.join(dataset_dir, "denovo_0.20_6.txt"))[0]
    target_sequence = get_sequences(os.path.join(dataset_dir, "target_sequence.txt"))[0]

    gap_indices = np.where(de_novo_sequence == '-')[0]
    incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
    correct_indices = np.where(de_novo_sequence == target_sequence)[0]
    incorrect_non_gaps = np.setdiff1d(incorrect_indices, gap_indices)
    print(f"Length of target: {len(target_sequence)}")
    print(f"Number of incorrect non-gaps: {len(incorrect_non_gaps)}")
    print(f"Number of gaps: {len(gap_indices)}")

    print_sequence(de_novo_sequence, "Protein Scaffold", incorrect_indices, correct_indices)

# =============================================================================
def test_original_protein_scaffold():

    ut.mkdir_if_not_exists(RESULTS_PATH)
    ut.mkdir_if_not_exists(DATAPATH)

    epochs = 300
    noise_percent = 0.25

    cwd = os.getcwd()

    # change to the location of the data for reading
    os.chdir(DATAPATH)
    de_novo_sequence = get_sequences("de_novo_sequence.txt")[0]
    target_sequence = get_sequences("target_sequence.txt")[0]
    alltrainingdata = get_sequences("training_sequences.txt")
    metadata = "training_sequences_metadata.csv"

    # then change to the results directory for writing
    os.chdir(RESULTS_PATH)

    sequences_to_train_on = [1000]
    # sequences_to_train_on = [100, 500, 1000, 2000]

    trainfilename = "cda_breakdown_breakdown_of_numtrain.csv"
    headers = ["Num Training Instances", "Full Accuracy", "Gap Accuracy", "Nongap Accuracy"]
    with open(trainfilename, "w+", encoding="utf-8", newline="") as f:
        train_writer = csv.writer(f)
        train_writer.writerow(headers)

    train_file = open(trainfilename, "a+", encoding="utf-8", newline="")
    train_writer = csv.writer(train_file)

    for numtrain in sequences_to_train_on:

        trainingdata = alltrainingdata[:numtrain]
        random.shuffle(trainingdata)

        incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices = np.where(de_novo_sequence == target_sequence)[0]

        fixer = ProteinScaffoldFixer(trainingdata, noise_percent=noise_percent, epochs=epochs)
        weights = fixer.train(verbose=1)

        hist = fixer.history.history
        training_loss = hist["loss"]
        validation_loss = hist["val_loss"]
        training_acc = hist["accuracy"]
        validation_acc = hist["val_accuracy"]

        headers = ["Training Accuracy", "Validation Accuracy", "Training Loss", "Validation Loss"]
        with open(f"autoencoder_results_on_original_{numtrain}.csv", "w", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(headers)
            writer.writerows(zip(training_acc, validation_acc, training_loss, validation_loss))

        predict_only_gaps = False
        pred = fixer.predict_sequence(de_novo_sequence, predict_only_gaps)

        incorrect_indices = np.where(target_sequence != pred)[0]
        correct_indices = np.where(target_sequence == pred)[0]
        indices_to_predict = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices_to_predict = np.intersect1d(correct_indices, indices_to_predict)

        gap_indices = np.where(de_novo_sequence == "-")[0]
        error_indices = np.setdiff1d(indices_to_predict, gap_indices)

        correct_gap_indices = np.intersect1d(gap_indices, correct_indices)
        correct_error_indices = np.intersect1d(error_indices, correct_indices)

        gap_acc = len(correct_gap_indices) / len(gap_indices)
        err_acc = len(correct_error_indices) / len(error_indices)
        full_acc = len(np.where(target_sequence == pred)[0]) / len(target_sequence)

        # print to console the prediction
        print_sequence(pred, f"Predicted for {numtrain}", incorrect_indices, correct_indices_to_predict)

        # write the final accuracies according to the number of training instances
        with open(trainfilename, "a+", encoding="utf-8", newline="") as f:
            train_writer = csv.writer(f)
            train_writer.writerow([numtrain, full_acc, gap_acc, err_acc])

        # and save the predictions as images for use in the paper
        ut.write_protein_scaffold_image(pred, incorrect_indices, correct_indices_to_predict, f"cda_predictions_on_original_{numtrain}")
    
    
    train_file.close()
    os.chdir(cwd)
    return weights

# =============================================================================
def test_generated_datasets():

    ut.mkdir_if_not_exists(RESULTS_PATH)
    ut.mkdir_if_not_exists(DATAPATH)

    epochs = 200
    noise_percent = 0.25

    cwd = os.getcwd()

    os.chdir(DATAPATH)
    target_sequence = get_sequences("target_sequence.txt")[0]

    sequences_to_train_on = np.array([1000])
    # sequences_to_train_on = np.array([100, 500, 1000, 2000])
    percent_missings = np.array([0.20, 0.30, 0.40])
    num_gaps = np.array([4, 6, 8, 10])

    resultsfilename = "cda_generated_accuracies.csv"

    os.chdir(RESULTS_PATH)
    with open(resultsfilename, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Percent Missing", "Num Gaps", "Training Instances", "Full Accuracy", "Gap Accuracy", "Non-gap Accuracy", "Most Similar Reference", "Least Similar Reference"])

    for percent_missing in percent_missings:
        for num_gap in num_gaps:

            os.chdir(DATAPATH)

            scaffold_sequence = get_sequences(f"denovo_{percent_missing:.2f}_{num_gap}.txt")[0]
            alltrainingdata = get_sequences(f"training_{percent_missing:.2f}_{num_gap}.txt")
            metadata = []

            with open(f"training_{percent_missing:.2f}_{num_gap}_metadata.csv", "r") as f:
                metadata = list(csv.reader(f))

            for num_train in sequences_to_train_on:

                most_similar = metadata[0][2]
                least_similar = metadata[num_train-1][2]

                trainingdata = alltrainingdata[:num_train]
                fixer = ProteinScaffoldFixer(trainingdata, noise_percent=noise_percent, epochs=epochs, early_stopping=True)
                fixer.train(verbose=1)

                predict_only_gaps = False
                pred = fixer.predict_sequence(scaffold_sequence, predict_only_gaps)

                incorrect_indices = np.where(target_sequence != pred)[0]
                correct_indices = np.where(target_sequence == pred)[0]

                indices_to_predict = np.where(scaffold_sequence != target_sequence)[0]
                correct_indices_to_predict = np.intersect1d(correct_indices, indices_to_predict)

                gap_indices = np.where(scaffold_sequence == "-")[0]
                error_indices = np.setdiff1d(indices_to_predict, gap_indices)

                correct_gap_indices = np.intersect1d(gap_indices, correct_indices)
                correct_error_indices = np.intersect1d(error_indices, correct_indices)

                gap_acc = len(correct_gap_indices) / len(gap_indices)
                err_acc = len(correct_error_indices) / len(error_indices)
                full_acc = len(np.where(target_sequence == pred)[0]) / len(target_sequence)

                # Print to console
                print(f"Gap Acc for CDA: {gap_acc}")
                print(f"Err Acc for CDA: {err_acc}")
                print(f"Full Acc for CDA: {full_acc}")
                print_sequence(pred, f"Results on gap={num_gap}, {percent_missing}, {num_train}", incorrect_indices, correct_indices_to_predict)

                # And save results 
                os.chdir(RESULTS_PATH)
                with open(resultsfilename, "a+", encoding="utf-8", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow([percent_missing, num_gap, num_train, full_acc, gap_acc, err_acc, most_similar, least_similar])

                # Also save the images for the paper
                ut.write_protein_scaffold_image(pred, incorrect_indices, correct_indices_to_predict, f"de_novo_{percent_missing:.2f}_{num_gap}_{num_train}")

    os.chdir(cwd)


In [8]:
test_original_protein_scaffold()

Epoch 1/300
27/27 [==============================] - 2s 34ms/step - loss: 1.8626 - accuracy: 0.5141 - val_loss: 1.4285 - val_accuracy: 0.5847
Epoch 2/300
27/27 [==============================] - 1s 25ms/step - loss: 1.4103 - accuracy: 0.5782 - val_loss: 1.3677 - val_accuracy: 0.5856
Epoch 3/300
27/27 [==============================] - 0s 18ms/step - loss: 1.3674 - accuracy: 0.5845 - val_loss: 1.3011 - val_accuracy: 0.5977
Epoch 4/300
27/27 [==============================] - 1s 20ms/step - loss: 1.2972 - accuracy: 0.5983 - val_loss: 1.2026 - val_accuracy: 0.6222
Epoch 5/300
27/27 [==============================] - 1s 21ms/step - loss: 1.1898 - accuracy: 0.6182 - val_loss: 1.0355 - val_accuracy: 0.6603
Epoch 6/300
27/27 [==============================] - 1s 22ms/step - loss: 1.0463 - accuracy: 0.6486 - val_loss: 0.8567 - val_accuracy: 0.7147
Epoch 7/300
27/27 [==============================] - 1s 28ms/step - loss: 0.8947 - accuracy: 0.6871 - val_loss: 0.6949 - val_accuracy: 0.7625
Epoch 

27/27 [==============================] - 1s 22ms/step - loss: 0.2167 - accuracy: 0.9424 - val_loss: 0.1545 - val_accuracy: 0.9643
Epoch 59/300
27/27 [==============================] - 1s 23ms/step - loss: 0.2144 - accuracy: 0.9428 - val_loss: 0.1542 - val_accuracy: 0.9646
Epoch 60/300
27/27 [==============================] - 1s 28ms/step - loss: 0.2134 - accuracy: 0.9431 - val_loss: 0.1529 - val_accuracy: 0.9645
Epoch 61/300
27/27 [==============================] - 1s 20ms/step - loss: 0.2134 - accuracy: 0.9432 - val_loss: 0.1522 - val_accuracy: 0.9646
Epoch 62/300
27/27 [==============================] - 1s 22ms/step - loss: 0.2117 - accuracy: 0.9435 - val_loss: 0.1516 - val_accuracy: 0.9652
Epoch 63/300
27/27 [==============================] - 1s 21ms/step - loss: 0.2108 - accuracy: 0.9435 - val_loss: 0.1519 - val_accuracy: 0.9647
Epoch 64/300
27/27 [==============================] - 1s 21ms/step - loss: 0.2094 - accuracy: 0.9444 - val_loss: 0.1511 - val_accuracy: 0.9647
Epoch 65/300

27/27 [==============================] - 1s 21ms/step - loss: 0.1775 - accuracy: 0.9525 - val_loss: 0.1361 - val_accuracy: 0.9690
Epoch 116/300
27/27 [==============================] - 1s 21ms/step - loss: 0.1764 - accuracy: 0.9530 - val_loss: 0.1350 - val_accuracy: 0.9688
Epoch 117/300
27/27 [==============================] - 1s 22ms/step - loss: 0.1771 - accuracy: 0.9524 - val_loss: 0.1350 - val_accuracy: 0.9691
Epoch 118/300
27/27 [==============================] - 1s 21ms/step - loss: 0.1774 - accuracy: 0.9527 - val_loss: 0.1350 - val_accuracy: 0.9693
Epoch 119/300
27/27 [==============================] - 1s 19ms/step - loss: 0.1757 - accuracy: 0.9531 - val_loss: 0.1346 - val_accuracy: 0.9696
Epoch 120/300
27/27 [==============================] - 1s 24ms/step - loss: 0.1749 - accuracy: 0.9535 - val_loss: 0.1341 - val_accuracy: 0.9692
Epoch 121/300
27/27 [==============================] - 1s 24ms/step - loss: 0.1758 - accuracy: 0.9529 - val_loss: 0.1352 - val_accuracy: 0.9692
Epoch 

Epoch 172/300
27/27 [==============================] - 1s 21ms/step - loss: 0.1614 - accuracy: 0.9564 - val_loss: 0.1274 - val_accuracy: 0.9717
Epoch 173/300
27/27 [==============================] - 1s 21ms/step - loss: 0.1608 - accuracy: 0.9566 - val_loss: 0.1280 - val_accuracy: 0.9715
Epoch 174/300
27/27 [==============================] - 1s 20ms/step - loss: 0.1599 - accuracy: 0.9567 - val_loss: 0.1265 - val_accuracy: 0.9714
Epoch 175/300
27/27 [==============================] - 1s 23ms/step - loss: 0.1599 - accuracy: 0.9568 - val_loss: 0.1264 - val_accuracy: 0.9715
Epoch 176/300
27/27 [==============================] - 1s 23ms/step - loss: 0.1597 - accuracy: 0.9566 - val_loss: 0.1263 - val_accuracy: 0.9717
Epoch 177/300
27/27 [==============================] - 1s 27ms/step - loss: 0.1598 - accuracy: 0.9569 - val_loss: 0.1265 - val_accuracy: 0.9719
Epoch 178/300
27/27 [==============================] - 1s 22ms/step - loss: 0.1602 - accuracy: 0.9566 - val_loss: 0.1277 - val_accuracy:

Epoch 229/300
27/27 [==============================] - 1s 22ms/step - loss: 0.1517 - accuracy: 0.9585 - val_loss: 0.1243 - val_accuracy: 0.9726
Epoch 230/300
27/27 [==============================] - 1s 20ms/step - loss: 0.1508 - accuracy: 0.9589 - val_loss: 0.1236 - val_accuracy: 0.9728
Epoch 231/300
27/27 [==============================] - 1s 19ms/step - loss: 0.1500 - accuracy: 0.9588 - val_loss: 0.1237 - val_accuracy: 0.9729
Epoch 232/300
27/27 [==============================] - 1s 20ms/step - loss: 0.1495 - accuracy: 0.9591 - val_loss: 0.1225 - val_accuracy: 0.9728
Epoch 233/300
27/27 [==============================] - 1s 22ms/step - loss: 0.1488 - accuracy: 0.9590 - val_loss: 0.1243 - val_accuracy: 0.9727
Epoch 234/300
27/27 [==============================] - 1s 24ms/step - loss: 0.1502 - accuracy: 0.9589 - val_loss: 0.1234 - val_accuracy: 0.9730
Epoch 235/300
27/27 [==============================] - 1s 25ms/step - loss: 0.1498 - accuracy: 0.9587 - val_loss: 0.1241 - val_accuracy:

Epoch 286/300
27/27 [==============================] - 1s 24ms/step - loss: 0.1443 - accuracy: 0.9599 - val_loss: 0.1202 - val_accuracy: 0.9740
Epoch 287/300
27/27 [==============================] - 1s 23ms/step - loss: 0.1441 - accuracy: 0.9602 - val_loss: 0.1207 - val_accuracy: 0.9741
Epoch 288/300
27/27 [==============================] - 1s 20ms/step - loss: 0.1431 - accuracy: 0.9602 - val_loss: 0.1210 - val_accuracy: 0.9739
Epoch 289/300
27/27 [==============================] - 1s 22ms/step - loss: 0.1430 - accuracy: 0.9603 - val_loss: 0.1208 - val_accuracy: 0.9737
Epoch 290/300
27/27 [==============================] - 1s 21ms/step - loss: 0.1434 - accuracy: 0.9602 - val_loss: 0.1214 - val_accuracy: 0.9740
Epoch 291/300
27/27 [==============================] - 1s 24ms/step - loss: 0.1436 - accuracy: 0.9598 - val_loss: 0.1215 - val_accuracy: 0.9739
Epoch 292/300
27/27 [==============================] - 1s 26ms/step - loss: 0.1441 - accuracy: 0.9597 - val_loss: 0.1227 - val_accuracy:

In [3]:
test_generated_datasets()

Epoch 1/200


2023-06-10 09:51:43.405748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 09:51:45.097786: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14759 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2023-06-10 09:51:45.098456: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14759 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-06-10 09:51:45.098924: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/tas

27/27 [==============================] - 6s 54ms/step - loss: 1.7091 - accuracy: 0.5321 - val_loss: 1.4330 - val_accuracy: 0.5738
Epoch 2/200
27/27 [==============================] - 1s 20ms/step - loss: 1.4086 - accuracy: 0.5757 - val_loss: 1.3886 - val_accuracy: 0.5776
Epoch 3/200
27/27 [==============================] - 1s 20ms/step - loss: 1.3495 - accuracy: 0.5864 - val_loss: 1.3057 - val_accuracy: 0.5973
Epoch 4/200
27/27 [==============================] - 1s 20ms/step - loss: 1.2515 - accuracy: 0.6039 - val_loss: 1.1790 - val_accuracy: 0.6262
Epoch 5/200
27/27 [==============================] - 1s 20ms/step - loss: 1.1227 - accuracy: 0.6287 - val_loss: 1.0142 - val_accuracy: 0.6684
Epoch 6/200
27/27 [==============================] - 1s 20ms/step - loss: 0.9616 - accuracy: 0.6668 - val_loss: 0.8315 - val_accuracy: 0.7350
Epoch 7/200
27/27 [==============================] - 1s 20ms/step - loss: 0.8057 - accuracy: 0.7131 - val_loss: 0.6925 - val_accuracy: 0.7879
Epoch 8/200
27/27 

Epoch 1/200
27/27 [==============================] - 3s 28ms/step - loss: 1.7581 - accuracy: 0.5330 - val_loss: 1.4364 - val_accuracy: 0.5730
Epoch 2/200
27/27 [==============================] - 1s 21ms/step - loss: 1.4031 - accuracy: 0.5775 - val_loss: 1.3674 - val_accuracy: 0.5822
Epoch 3/200
27/27 [==============================] - 1s 22ms/step - loss: 1.3281 - accuracy: 0.5923 - val_loss: 1.2817 - val_accuracy: 0.5999
Epoch 4/200
27/27 [==============================] - 1s 23ms/step - loss: 1.2276 - accuracy: 0.6082 - val_loss: 1.1501 - val_accuracy: 0.6320
Epoch 5/200
27/27 [==============================] - 1s 23ms/step - loss: 1.0938 - accuracy: 0.6375 - val_loss: 0.9896 - val_accuracy: 0.6773
Epoch 6/200
27/27 [==============================] - 1s 22ms/step - loss: 0.9412 - accuracy: 0.6742 - val_loss: 0.8359 - val_accuracy: 0.7177
Epoch 7/200
27/27 [==============================] - 1s 22ms/step - loss: 0.8006 - accuracy: 0.7119 - val_loss: 0.7086 - val_accuracy: 0.7743
Epoch 

27/27 [==============================] - 1s 21ms/step - loss: 0.1603 - accuracy: 0.9574 - val_loss: 0.2648 - val_accuracy: 0.9398
Epoch 116/200
27/27 [==============================] - 1s 23ms/step - loss: 0.1596 - accuracy: 0.9577 - val_loss: 0.2689 - val_accuracy: 0.9402
Epoch 117/200
27/27 [==============================] - 1s 22ms/step - loss: 0.1601 - accuracy: 0.9576 - val_loss: 0.2646 - val_accuracy: 0.9401
Epoch 118/200
27/27 [==============================] - 1s 21ms/step - loss: 0.1600 - accuracy: 0.9576 - val_loss: 0.2661 - val_accuracy: 0.9402
Epoch 119/200
27/27 [==============================] - 1s 22ms/step - loss: 0.1601 - accuracy: 0.9575 - val_loss: 0.2623 - val_accuracy: 0.9409
Epoch 120/200
27/27 [==============================] - 1s 23ms/step - loss: 0.1578 - accuracy: 0.9583 - val_loss: 0.2641 - val_accuracy: 0.9404
Epoch 121/200
27/27 [==============================] - 1s 22ms/step - loss: 0.1577 - accuracy: 0.9580 - val_loss: 0.2651 - val_accuracy: 0.9405
Epoch 

...

1/1 [==============================] - 0s 326ms/step
Gap Acc for CDA: 0.19607843137254902
Err Acc for CDA: 0.6764705882352942
Full Acc for CDA: 0.5794392523364486
Results on gap=6, 0.4, 1000
D I Q M T Q S P S S L S A S V G D R V T I T C Q A S Q D I S N Y L N W Y Q Q K P
G K A P K L L I Y D A S N L E P G V P S R F S G S G S G T D F T L T I S S L Q P
E D F A T Y Y C Q Q Y N S Y P Y T F G Q G T K L E I K R T V A A P S V F I F P P
S D E Q L K S G T A S V L L L L L K K S S S S S S S F S P P P P P P A V V V V G
D A V V D D D I D I N S N N Y Y Y L N R R A A G A A E L L D Y M A Y Y Y S L S G
A A A A L T Y Y A A Q G D D
Epoch 1/200
27/27 [==============================] - 4s 53ms/step - loss: 1.6245 - accuracy: 0.5936 - val_loss: 1.1467 - val_accuracy: 0.6617
Epoch 2/200
27/27 [==============================] - 1s 19ms/step - loss: 1.1114 - accuracy: 0.6678 - val_loss: 1.0586 - val_accuracy: 0.6724
Epoch 3/200
27/27 [==============================] - 1s 20ms/step - loss: 0.9963 - accuracy: 0.69

27/27 [==============================] - 1s 20ms/step - loss: 0.1148 - accuracy: 0.9702 - val_loss: 0.1524 - val_accuracy: 0.9668
Epoch 99/200
27/27 [==============================] - 1s 20ms/step - loss: 0.1136 - accuracy: 0.9708 - val_loss: 0.1498 - val_accuracy: 0.9673
Epoch 100/200
27/27 [==============================] - 1s 19ms/step - loss: 0.1125 - accuracy: 0.9709 - val_loss: 0.1525 - val_accuracy: 0.9670
Epoch 101/200
27/27 [==============================] - 1s 20ms/step - loss: 0.1138 - accuracy: 0.9705 - val_loss: 0.1497 - val_accuracy: 0.9670
Epoch 102/200
27/27 [==============================] - 1s 20ms/step - loss: 0.1129 - accuracy: 0.9710 - val_loss: 0.1503 - val_accuracy: 0.9673
Epoch 103/200
27/27 [==============================] - 1s 20ms/step - loss: 0.1122 - accuracy: 0.9710 - val_loss: 0.1498 - val_accuracy: 0.9677
Epoch 104/200
27/27 [==============================] - 1s 20ms/step - loss: 0.1118 - accuracy: 0.9711 - val_loss: 0.1489 - val_accuracy: 0.9681
Epoch 1

Epoch 155/200
27/27 [==============================] - 1s 21ms/step - loss: 0.0973 - accuracy: 0.9741 - val_loss: 0.1390 - val_accuracy: 0.9709
Epoch 156/200
27/27 [==============================] - 1s 20ms/step - loss: 0.0964 - accuracy: 0.9741 - val_loss: 0.1402 - val_accuracy: 0.9706
Epoch 157/200
27/27 [==============================] - 1s 20ms/step - loss: 0.0969 - accuracy: 0.9743 - val_loss: 0.1430 - val_accuracy: 0.9713
Epoch 158/200
27/27 [==============================] - 1s 21ms/step - loss: 0.0960 - accuracy: 0.9745 - val_loss: 0.1383 - val_accuracy: 0.9709
Epoch 159/200
1/1 [==============================] - 0s 334ms/step
Gap Acc for CDA: 0.3137254901960784
Err Acc for CDA: 0.6470588235294118
Full Acc for CDA: 0.6542056074766355
Results on gap=8, 0.4, 1000
D I Q M T Q S P S S L S A S V G D R V T I T C Q A S Q D I S N Y L A W Y Q Q K P
G K A P K L L I Y D A S N L E T G V P S R F S G S G S G T D F T F T I S S L Q P
E D F A T Y Y C L Q H N S Y P R T F G Q G T K V D I K R T A 

27/27 [==============================] - 1s 24ms/step - loss: 0.2074 - accuracy: 0.9430 - val_loss: 0.1663 - val_accuracy: 0.9628
Epoch 96/200
27/27 [==============================] - 1s 24ms/step - loss: 0.2060 - accuracy: 0.9434 - val_loss: 0.1656 - val_accuracy: 0.9628
Epoch 97/200
27/27 [==============================] - 1s 23ms/step - loss: 0.2049 - accuracy: 0.9432 - val_loss: 0.1661 - val_accuracy: 0.9630
Epoch 98/200
27/27 [==============================] - 1s 23ms/step - loss: 0.2047 - accuracy: 0.9436 - val_loss: 0.1658 - val_accuracy: 0.9632
Epoch 99/200
27/27 [==============================] - 1s 22ms/step - loss: 0.2025 - accuracy: 0.9441 - val_loss: 0.1645 - val_accuracy: 0.9635
Epoch 100/200
27/27 [==============================] - 1s 22ms/step - loss: 0.2028 - accuracy: 0.9443 - val_loss: 0.1650 - val_accuracy: 0.9632
Epoch 101/200
27/27 [==============================] - 1s 21ms/step - loss: 0.2025 - accuracy: 0.9442 - val_loss: 0.1633 - val_accuracy: 0.9632
Epoch 102/

27/27 [==============================] - 1s 22ms/step - loss: 0.1830 - accuracy: 0.9488 - val_loss: 0.1554 - val_accuracy: 0.9655
Epoch 153/200
27/27 [==============================] - 1s 21ms/step - loss: 0.1808 - accuracy: 0.9497 - val_loss: 0.1543 - val_accuracy: 0.9655
Epoch 154/200
27/27 [==============================] - 1s 22ms/step - loss: 0.1812 - accuracy: 0.9493 - val_loss: 0.1538 - val_accuracy: 0.9659
Epoch 155/200
27/27 [==============================] - 1s 22ms/step - loss: 0.1809 - accuracy: 0.9494 - val_loss: 0.1532 - val_accuracy: 0.9658
Epoch 156/200
27/27 [==============================] - 1s 23ms/step - loss: 0.1814 - accuracy: 0.9493 - val_loss: 0.1536 - val_accuracy: 0.9657
Epoch 157/200
27/27 [==============================] - 1s 24ms/step - loss: 0.1801 - accuracy: 0.9498 - val_loss: 0.1540 - val_accuracy: 0.9657
Epoch 158/200
27/27 [==============================] - 1s 22ms/step - loss: 0.1792 - accuracy: 0.9501 - val_loss: 0.1534 - val_accuracy: 0.9659
Epoch 

In [3]:
test_original_protein_scaffold()

2023-06-11 16:48:48.242015: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 16:48:49.934996: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14759 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2023-06-11 16:48:49.935610: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14759 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-06-11 16:48:49.936062: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/tas

Epoch 1/10


2023-06-11 16:48:52.473834: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-06-11 16:48:53.752564: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f2b61f68f10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-11 16:48:53.752586: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-06-11 16:48:53.752590: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Quadro RTX 5000, Compute Capability 7.5
2023-06-11 16:48:53.752593: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (2): Quadro RTX 5000, Compute Capability 7.5
2023-06-11 16:48:53.752596: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (3): Quadro RTX 5000, Compute Capability 7.5
2023-06-11 16:48:53.756227: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabl

27/27 [==============================] - 6s 56ms/step - loss: 1.8859 - accuracy: 0.4828 - val_loss: 1.4059 - val_accuracy: 0.5806
Epoch 2/10
27/27 [==============================] - 1s 20ms/step - loss: 1.3914 - accuracy: 0.5825 - val_loss: 1.3572 - val_accuracy: 0.5843
Epoch 3/10
27/27 [==============================] - 1s 19ms/step - loss: 1.3332 - accuracy: 0.5911 - val_loss: 1.2839 - val_accuracy: 0.6023
Epoch 4/10
27/27 [==============================] - 1s 20ms/step - loss: 1.2667 - accuracy: 0.6044 - val_loss: 1.1870 - val_accuracy: 0.6245
Epoch 5/10
27/27 [==============================] - 1s 20ms/step - loss: 1.1619 - accuracy: 0.6235 - val_loss: 1.0244 - val_accuracy: 0.6650
Epoch 6/10
27/27 [==============================] - 1s 21ms/step - loss: 1.0001 - accuracy: 0.6633 - val_loss: 0.7961 - val_accuracy: 0.7312
Epoch 7/10
27/27 [==============================] - 1s 20ms/step - loss: 0.8350 - accuracy: 0.7059 - val_loss: 0.6303 - val_accuracy: 0.7932
Epoch 8/10
27/27 [======

In [5]:
print(len(weights))

12


In [6]:
for i in weights:
    print(i.shape)

(10, 24, 46)
(46,)
(9, 46, 90)
(90,)
(5, 90, 160)
(160,)
(5, 90, 160)
(90,)
(9, 46, 90)
(46,)
(5, 46, 24)
(24,)


In [7]:
def calculate_conv1d_output(input_size, padding, strides, kernel_size):
    if padding == 'same':
        output_size = input_size
    else:
        output_size = input_size - kernel_size + 1
    output_size = (output_size - 1) // strides + 1
    return output_size


def calculate_maxpool1d_output(input_size, pool_size, strides):
    output_size = (input_size - pool_size) // strides + 1
    return output_size

In [8]:
calculate_conv1d_output((214*1),'same',1,5)

214

In [19]:
calculate_maxpool1d_output((214*1),,2)

100

In [34]:
def calculate_conv_output(input_size, filter_size, padding, stride=1):
    output_size = (((input_size + 2*padding - filter_size) / stride) + 1) 
    return int(output_size)

In [36]:
calculate_conv_output((224*1),5,0,1)

220

In [ ]:
 (((n+2p-f)/s)+1)*(((n+2p-f)/s)+1)

In [17]:
p = (32-1)/2
p

15.5

In [5]:
string_mine = 'DIQRTQSPSSLSASVGOORVTTTCRASQN 1 DK ELNWEQQKPGRAPKLL J ENTRALQTGVPSRF SGSGSCTDF TF TISSLOPE DI ATYYCL QHISRPRTF GQGTKVE IKRTVAAPSVF IF PPSOEQLKSGTASVVCL L NNF YPREAKVQWKVDNAL QSGNSQESVTEQOSKDSTYSL SSTL TL SKADYEKHKVYACEVTHQGL SSPVTK SF NRGEC'


In [6]:
string_mine = string_mine.replace(' ','')
string_mine

'DIQRTQSPSSLSASVGOORVTTTCRASQN1DKELNWEQQKPGRAPKLLJENTRALQTGVPSRFSGSGSCTDFTFTISSLOPEDIATYYCLQHISRPRTFGQGTKVEIKRTVAAPSVFIFPPSOEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQOSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC'